In [1]:
import numpy as np
import nolds
import scipy.io
import mne
import pandas as pd
import seaborn as sns
import pyqtgraph as pg
from scipy.io import loadmat
from scipy import stats
import matplotlib.pyplot as plt
import antropy as ent
from attractors import Attractor
from antropy import higuchi_fd
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.computation import RQAComputation
from pyrqa.time_series import TimeSeries
from pyrqa.result import RQAResult
from pyrqa.opencl import OpenCL
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.settings import Settings
from pyrqa.time_series import TimeSeries
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.settings import Settings
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.image_generator import ImageGenerator
import numpy as np
import matplotlib.pyplot as plt
from antropy import higuchi_fd

In [4]:
# Load data
data = loadmat('/home/vincent/AAA_projects/MVCS/Neuroscience/downsampled/EEG_DS_Struct_0101.mat')

In [5]:
print(type(data['DSamp']))  # check the data type
print(data['DSamp'].shape)  # check the shape of the data
print(data['DSamp'])  # print the data

<class 'numpy.ndarray'>
(1, 1)
[[(array([[(array([[20.428]]), array([[20429]], dtype=uint16), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64)),
          (array([[619.442]]), array([[619443]], dtype=int32), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64)),
          (array([[619.499]]), array([[619500]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3')),
          (array([[654.746]]), array([[654747]], dtype=int32), array(['0032'], dtype='<U4'), array([[32]], dtype=uint8), array(['Stim Stop'], dtype='<U9'), array([], shape=(1, 0), dtype=float64)),
          (array([[770.515]]), array([[770516]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3'))

In [78]:
import pandas as pd

# URL to the web page with the table
url = "https://www.nature.com/articles/s41597-021-01046-y/tables/2"

# Read the HTML and parse the tables
tables = pd.read_html(url)

# Print the first table
print(tables[0])

   Sub# Session File Num F0 M0 P0 F5 M5 P5 F30 M30 P30 Stim Type          \
   Sub# Session File Num F0 M0 P0 F5 M5 P5 F30 M30 P30   Block 1 Block 2   
0     1       1      101  1  Â  Â  Â  Â  Â   7   8   Â       M30     F30   
1     1       2      102  1  Â  Â  Â  Â  Â   7   8   Â       M30     F30   
2     1       3      103  Â  Â  3  Â  Â  6   Â   Â   9       P30      P0   
3     1       4      104  Â  2  Â  4  5  Â   Â   Â   Â        F5      M5   
4     1       5      105  Â  2  Â  4  5  Â   Â   Â   Â        F5      M5   
5     1       6      106  Â  Â  3  Â  Â  6   Â   Â   9       P30      P0   
6     2       1      201  Â  Â  Â  4  Â  Â   7   Â   9    F30â¦      F5   
7     2       2      202  Â  Â  3  Â  5  Â   Â   8   Â        P0      M5   
8     3       1      301  Â  2  3  Â  Â  Â   7   Â   Â        M0      P0   
9     3       2      302  Â  Â  Â  4  Â  6   Â   Â   9        F5      P5   
10    3       3      303  1  Â  Â  Â  5  Â   Â   8   Â        M5      F0   
11    4     

In [79]:
def fetch_stimulation_amplitude(sub_num):
    # assuming sub_num is a unique identifier for each subject
    session_data = df[df[('Sub#', 'Sub#')] == sub_num]
    amplitudes = {
        'Block 1': session_data[('Stim Amplitude (mA)', 'Block 1')].values,
        'Block 2': session_data[('Stim Amplitude (mA)', 'Block 2')].values,
        'Block 3': session_data[('Stim Amplitude (mA)', 'Block 3')].values,
    }
    return amplitudes

In [80]:
def fetch_stimulation_amplitude(sub_num, session_num):
    # assuming sub_num is a unique identifier for each subject
    # assuming session_num is a unique identifier for each session
    session_data = df[(df[('Sub#', 'Sub#')] == sub_num) & (df[('Session', 'Session')] == session_num)]
    amplitudes = {
        'Block 1': session_data[('Stim Amplitude (mA)', 'Block 1')].values[0],
        'Block 2': session_data[('Stim Amplitude (mA)', 'Block 2')].values[0],
        'Block 3': session_data[('Stim Amplitude (mA)', 'Block 3')].values[0],
    }
    return amplitudes

In [81]:
DSamp = data['DSamp']

triggers = DSamp[0][0][0]
EEGdata = DSamp[0][0][1]
fs = DSamp[0][0][2][0][0] 
fsOld = DSamp[0][0][3][0][0] 
time = DSamp[0][0][4][0]
label = DSamp[0][0][5]
nchan = DSamp[0][0][6][0][0]
rate = DSamp[0][0][7][0][0]
npt = DSamp[0][0][8][0][0]
Subj = DSamp[0][0][9][0]
ptrackerPerf = DSamp[0][0][10]
ptrackerTime = DSamp[0][0][11]
ptrackerfs = DSamp[0][0][12][0][0]

In [45]:
triggers = DSamp[0][0][0]
print("Triggers: ", triggers)

EEGdata = DSamp[0][0][1]
print("EEGdata: ", EEGdata)

fs = DSamp[0][0][2][0][0] 
print("fs: ", fs)

fsOld = DSamp[0][0][3][0][0] 
print("fsOld: ", fsOld)

time = DSamp[0][0][4][0]
print("Time: ", time)

label = DSamp[0][0][5]
print("Label: ", label)

nchan = DSamp[0][0][6][0][0]
print("nchan: ", nchan)

rate = DSamp[0][0][7][0][0]
print("Rate: ", rate)

npt = DSamp[0][0][8][0][0]
print("npt: ", npt)

Subj = DSamp[0][0][9][0]
print("Subj: ", Subj)

ptrackerPerf = DSamp[0][0][10]
print("PtrackerPerf: ", ptrackerPerf)

ptrackerTime = DSamp[0][0][11]
print("PtrackerTime: ", ptrackerTime)

ptrackerfs = DSamp[0][0][12][0][0]
print("Ptrackerfs: ", ptrackerfs)

Triggers:  [[(array([[20.428]]), array([[20429]], dtype=uint16), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64))
  (array([[619.442]]), array([[619443]], dtype=int32), array(['0002'], dtype='<U4'), array([[2]], dtype=uint8), array(['Block Start'], dtype='<U11'), array([], shape=(1, 0), dtype=float64))
  (array([[619.499]]), array([[619500]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3'))
  (array([[654.746]]), array([[654747]], dtype=int32), array(['0032'], dtype='<U4'), array([[32]], dtype=uint8), array(['Stim Stop'], dtype='<U9'), array([], shape=(1, 0), dtype=float64))
  (array([[770.515]]), array([[770516]], dtype=int32), array(['0016'], dtype='<U4'), array([[16]], dtype=uint8), array(['Stim Start'], dtype='<U10'), array(['M30'], dtype='<U3'))
  (array([[805.571]]), array([[805572]], dtype=int32), array(['0

## Other topics to explore: Cable Theory-Based Models, Compartmental Neuron Models, Network Models, Mean-Field Models, Detailed Biophysical Models, Graph Theory, Topological Data Analysis

## Next Modules to make: Convolutional Neural Network, Hjorth Coefficients, Petrosian Fractal Dimension, Band Power, Spectral Entropy, Discrete Wavelet Transform, Differential Asymmetry, Magnitude Squared Coherence Estimate